Plot the data...

In [ ]:
data_dir = ".\\data\\"

#filename = "2018-06-20-10-44-11.tdms"
#keyence_filename= "2018-06-20-10-44-18-KEYENCE.tdms"

#filename = "2018-07-12-18-36-41.tdms"
#keyence_filename= "2018-07-12-18-36-56-KEYENCE.tdms"

#filename = "2018-07-12-18-51-08.tdms"
#keyence_filename= "2018-07-12-18-51-17-KEYENCE.tdms"

#filename = "2018-07-17-10-17-29.tdms"
filename = "2018-07-17-09-45-52.tdms"
keyence_filename = None

In [ ]:
import pickle

if keyence_filename is not None:
    pickle_file = data_dir + "_".join([filename, keyence_filename, "proc_data.p"])
else:
    pickle_file = data_dir + "_".join([filename, "proc_data.p"])

try:
    data = pickle.load(open(pickle_file, 'rb'))
    print("Loaded the data from " + pickle_file)
except FileNotFoundError:
    print("File not found, load the data again!")

In [ ]:
# Don't show the filename attribute - it's already the title!!
del data['info']['Filename: ']

# Pop the info out, it can't be processed...
info = data.pop("info")

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

import colorlover as cl
colors = cl.scales['9']['qual']['Set1']
colors.pop(5)  # yellow, euwww

plotter = Scatter
#plotter = Scattergl

plots = [
    plotter(x=data['position']['time'],  y=data['position']['data'],  
            name="Stage Position", line={'color':colors[1]}),
    plotter(x=data['set_point']['time'], y=data['set_point']['data'], 
            name="Set Point", line={'color':colors[0], 'shape':'hv'}),
    
    plotter(x=data['set_point']['time'], y=data['set_point']['trans'],   
            name="Transition", yaxis='y4', line={'color':colors[2]}),
    
    plotter(x=data['temperature']['time'], y=data['temperature']['stage'], 
            name="Stage",   yaxis='y2', line={'color':colors[1]}), 
    plotter(x=data['temperature']['time'], y=data['temperature']['base'],  
            name="Base",    yaxis='y2', line={'color':colors[0]}),
    plotter(x=data['temperature']['time'], y=data['temperature']['amb'],   
            name="Ambient", yaxis='y2', line={'color':colors[2]}),
    
    plotter(x=data['temperature']['time'], y=data['temperature']['stage']-data['temperature']['amb'], 
            name="Stage-Ambient",   yaxis='y3', line={'color':colors[3]}), 
    plotter(x=data['temperature']['time'], y=data['temperature']['base']-data['temperature']['amb'],  
            name="Base-Ambient",    yaxis='y3', line={'color':colors[4]}),
    ]

layout = Layout(
    width = 800,
    height = 800,
    title = filename,
    xaxis = dict(
        title = "Time",
        #rangeslider=dict(thickness=0.05),
        anchor='y2'
    ),
    yaxis = dict(
        title = "Stage Position (mm)",
        domain = [0.5, 1]
    ),
    yaxis2 = dict(
        title = "Temperature (°C)",
        domain = [0, 0.2]
    ),
    yaxis3 = dict(
        title = "Temperature diff (°C)",
        overlaying='y2',
        side='right',
        scaleanchor = 'y2',
    ),
    yaxis4 = dict(
        title = "Transition time (s)",
        domain = [0.25, 0.45]
    ),
    showlegend = True,
    legend = dict(
        x = 1.1,
        y = 1,
        orientation="v"
    ),
    annotations=[
        dict(
            x=1.2,
            y=0.4,
            align='left',
            showarrow=False,
            text='<br>'.join(info.keys()),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.45,
            y=0.4,
            align='left',
            showarrow=False,
            text='<br>'.join(info.values()),
            xref='paper',
            yref='paper'
        ),
    ]
)

fig = Figure(data=plots, layout=layout)
iplot(fig, filename=filename)

In [ ]:
marker = dict(symbol='dot', maxdisplayed=100, line=dict(width=0))

if keyence_filename is not None:
    plots = plots[:8]
    plots.append(plotter(x=data['keyence']['time'], y=data['keyence']['data1'], name="Sensor 1", yaxis='y5', mode='markers', marker=marker, line={'color':colors[5]}))
    plots.append(plotter(x=data['keyence']['time'], y=data['keyence']['data2'], name="Sensor 2", yaxis='y5', mode='markers', marker=marker, line={'color':colors[6]}))

    layout['yaxis5'] = dict(
            title = "Keyence Position (mm)",
            scaleanchor = "y",
            overlaying='y',
            side='right',
        )

    #layout['yaxis']['range']  = [-12.5, -7.5]
    #layout['yaxis5']['range'] = [0.5, -4.5]

    layout['title'] = "{} + {}".format(filename, keyence_filename)

    fig = Figure(data=plots, layout=layout)
    iplot(fig, filename=pickle_file)

In [ ]:
new_plots = plots[:1] + plots[8:]
hoverformat = '.4f'

keyence_mean = 8.0

new_plots.append(plotter(x=data['position']['time'], 
                     y=data['position']['data_set_point_data'], 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6',
                     line=dict(color=colors[1])
            ))
new_plots.append(Histogram( 
                     y=data['position']['data_set_point_data'], 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6', xaxis='x2',
                     marker=dict(color=colors[1]),
                     opacity = 0.4
            ))


#new_plots.append(plotter(x=data['keyence']['time'], 
#                     y=data['keyence']['data1_set_point_data'], 
#                     name="Sensor 1 - Set point", legendgroup='data1_set_point_data',
#                     yaxis='y6',
#                     line=dict(color=colors[5])
#            ))
#new_plots.append(Histogram( 
#                     y=data['keyence']['data1_set_point_data'], 
#                     name="Sensor 1 - Set point", legendgroup='data1_set_point_data',
#                     yaxis='y6', xaxis='x2',
#                     marker=dict(color=colors[5]),
#                     opacity = 0.4
#            ))

#new_plots.append(plotter(x=data['keyence']['time'], 
#                     y=data['keyence']['data2_set_point_data'], 
#                     name="Sensor 2 - Set point", legendgroup='data2_set_point_data',
#                     yaxis='y6',
#                     line=dict(color=colors[6])
#            ))
#new_plots.append(Histogram( 
#                     y=data['keyence']['data2_set_point_data'], 
#                     name="Sensor 2 - Set point", legendgroup='data2_set_point_data',
#                     yaxis='y6', xaxis='x2',
#                     marker=dict(color=colors[6]),
#                     opacity = 0.4
#            ))

if keyence_filename is not None:
    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=data['keyence']['data1_position_data'] - keyence_mean, 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6',
                         line=dict(color=colors[5])
                ))
    new_plots.append(Histogram( 
                         y=data['keyence']['data1_position_data'] - keyence_mean, 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[5]),
                         opacity = 0.4
                ))

    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=data['keyence']['data2_position_data'] - keyence_mean, 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6',
                         line=dict(color=colors[6])
                ))
    new_plots.append(Histogram( 
                         y=data['keyence']['data2_position_data'] - keyence_mean, 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[6]),
                         opacity = 0.4
                ))

layout['yaxis6'] = dict(
        title = "Difference (mm)",
        overlaying='y4',
        side='left',
        hoverformat=hoverformat
    )
layout['yaxis4']['side'] = 'right'
layout['xaxis2'] = dict(
        title = "Log Prob",
        anchor = "y6",
        domain = [0.75, 1],
        type = "log",
        hoverformat=hoverformat,
        tickangle = -90,
    )

layout['xaxis']['domain'] = [0, 0.7]
layout['yaxis6']['domain'] = [0, 0.45]
layout['yaxis4']['domain'] = [0.55, 1]

try:
    del layout['annotations']
except KeyError:
    pass

layout['legend'] = dict(
        x = 1.1,
        y = 1,
        orientation="v"
    )

layout['barmode']='overlay'

fig = Figure(data=new_plots, layout=layout)
iplot(fig, filename="{} + {} - Difference".format(filename, keyence_filename))

In [ ]:
import numpy as np

new_plots = plots[:2] + plots[8:10]
hoverformat = '.4f'
mode="lines"#+markers"

new_plots.append(plotter(x=data['position']['time'], 
                     y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                         mask=data['position']['mask_set_point']), 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6', mode=mode,
                     line=dict(color=colors[1])
            ))
new_plots.append(Histogram( 
                     y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                         mask=data['position']['mask_set_point']), 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6', xaxis='x2',
                     marker=dict(color=colors[1]),
                     opacity = 0.4
            ))

if keyence_filename is not None:
    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=np.ma.MaskedArray(data['keyence']['data1_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6', mode=mode,
                         line=dict(color=colors[5])
                ))
    new_plots.append(Histogram( 
                         y=np.ma.MaskedArray(data['keyence']['data1_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[5]),
                         opacity = 0.4
                ))

    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=np.ma.MaskedArray(data['keyence']['data2_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6', mode=mode,
                         line=dict(color=colors[6])
                ))
    new_plots.append(Histogram( 
                         y=np.ma.MaskedArray(data['keyence']['data2_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[6]),
                         opacity = 0.4
                ))

layout['yaxis6'] = dict(
        title = "Difference (mm)",
        overlaying='y4',
        side='left',
        hoverformat=hoverformat
    )
layout['yaxis4']['side'] = 'right'
layout['xaxis2'] = dict(
        title = "Log Prob",
        anchor = "y6",
        domain = [0.75, 1],
        type = "log",
        hoverformat=hoverformat,
        tickangle = -90,
    )

layout['xaxis']['domain'] = [0, 0.7]
layout['yaxis6']['domain'] = [0, 0.45]
layout['yaxis4']['domain'] = [0.55, 1]

layout['xaxis']['anchor'] = 'y6'

layout['legend'] = dict(
        x = 1.1,
        y = 1,
        orientation="v"
    )

layout['barmode']='overlay'

fig = Figure(data=new_plots, layout=layout)
iplot(fig, filename="{} + {} - Difference".format(filename, keyence_filename))

In [ ]:
new_plots = plots[3:8]
if keyence_filename is not None:
    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=np.ma.MaskedArray(data['keyence']['data1_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6', mode=mode,
                         line=dict(color=colors[5])
                ))
    new_plots.append(Histogram( 
                         y=np.ma.MaskedArray(data['keyence']['data1_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[5]),
                         opacity = 0.4
                ))

    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=np.ma.MaskedArray(data['keyence']['data2_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6', mode=mode,
                         line=dict(color=colors[6])
                ))
    new_plots.append(Histogram( 
                         y=np.ma.MaskedArray(data['keyence']['data2_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[6]),
                         opacity = 0.4
                ))
else:
    new_plots.append(plotter(x=data['position']['time'], 
                         y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                             mask=data['position']['mask_set_point']), 
                         name="Pos - Set point", legendgroup='data_set_point_data',
                         yaxis='y6', mode=mode,
                         line=dict(color=colors[1])
                ))
    new_plots.append(Histogram( 
                         y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                             mask=data['position']['mask_set_point']), 
                         name="Pos - Set point", legendgroup='data_set_point_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[1]),
                         opacity = 0.4
                ))



layout['yaxis6']['domain'] = [0, 0.45]
layout['yaxis2']['domain'] = [0.55, 1]

fig = Figure(data=new_plots, layout=layout)
iplot(fig, filename="{} + {} - Diff & Temp".format(filename, keyence_filename))

In [ ]:
from data_tools import *

limits = [data['set_point']['time'][0], data['set_point']['time'][-1]]

print("Using limits:")
pprint(limits)

# Truncate the data
data = dict(zip(data.keys(), [truncate(limits, data[k]) for k in data]))

print("Truncated dataset:")
for k in data:
    print("{}: \t{} points".format(k, len(data[k][[*data[k]][0]])))

In [ ]:
import numpy as np

new_plots = [
    plotter(x=data['position']['time'],  y=data['position']['data'],  
            name="Stage Position", line={'color':colors[1]}),
    plotter(x=data['set_point']['time'], y=data['set_point']['data'], 
            name="Set Point", line={'color':colors[0], 'shape':'hv'})
]

    
hoverformat = '.4f'
mode="lines"#+markers"

new_plots.append(plotter(x=data['position']['time'], 
                     y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                         mask=data['position']['mask_set_point']), 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6', mode=mode,
                     line=dict(color=colors[1])
            ))
new_plots.append(Histogram( 
                     y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                         mask=data['position']['mask_set_point']), 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6', xaxis='x2',
                     marker=dict(color=colors[1]),
                     opacity = 0.4
            ))

if keyence_filename is not None:
    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=np.ma.MaskedArray(data['keyence']['data1_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6', mode=mode,
                         line=dict(color=colors[5])
                ))
    new_plots.append(Histogram( 
                         y=np.ma.MaskedArray(data['keyence']['data1_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 1 - Position", legendgroup='data1_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[5]),
                         opacity = 0.4
                ))

    new_plots.append(plotter(x=data['keyence']['time'], 
                         y=np.ma.MaskedArray(data['keyence']['data2_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6', mode=mode,
                         line=dict(color=colors[6])
                ))
    new_plots.append(Histogram( 
                         y=np.ma.MaskedArray(data['keyence']['data2_position_data'] - keyence_mean, 
                                             mask=data['keyence']['mask_set_point']), 
                         name="Sensor 2 - Position", legendgroup='data2_position_data',
                         yaxis='y6', xaxis='x2',
                         marker=dict(color=colors[6]),
                         opacity = 0.4
                ))

layout['yaxis6'] = dict(
        title = "Difference (mm)",
        overlaying='y4',
        side='left',
        hoverformat=hoverformat
    )
layout['yaxis4']['side'] = 'right'
layout['xaxis2'] = dict(
        title = "Log Prob",
        anchor = "y6",
        domain = [0.75, 1],
        type = "log",
        hoverformat=hoverformat,
        tickangle = -90,
    )

layout['xaxis']['domain'] = [0, 0.7]
layout['yaxis6']['domain'] = [0, 0.45]
layout['yaxis4']['domain'] = [0.55, 1]

layout['xaxis']['anchor'] = 'y6'

layout['legend'] = dict(
        x = 1.1,
        y = 1,
        orientation="v"
    )

layout['barmode']='overlay'

fig = Figure(data=new_plots, layout=layout)
iplot(fig, filename="{} + {} - Difference".format(filename, keyence_filename))